In [1]:
import pandas as pd
from datetime import datetime

In [2]:
exchange_info = {
    "LN": {
        "start": "09:30",
        "end": "17:30",
        "exchange_rate": 1.2853,
        "region": "WE",
        "name": "London"
    },
    "US": {
        "start": "13:30",
        "end": "21:30",
        "exchange_rate": 1,
        "region": "NA",
        "name": "United States"
    }
}

In [3]:
exchange_info

{'LN': {'start': '09:30',
  'end': '17:30',
  'exchange_rate': 1.2853,
  'region': 'WE',
  'name': 'London'},
 'US': {'start': '13:30',
  'end': '21:30',
  'exchange_rate': 1,
  'region': 'NA',
  'name': 'United States'}}

In [4]:
example_df = pd.DataFrame({
    "exchange": ["LN", "LN", "US", "LN", "US"],
    "start_time": ["09:34", "08:59", "10:44", "12:44", "9:34"],
    "end_time": ["09:39", "21:59", "10:48", "12:34", "11:34"],
    "price": [452, 145, 34, 12, 4],
    "avg_price": [45, 2, 45, 12, 5]
})

---

In [5]:
example_df.loc[5] = ["BZ", "05:44", "09:55", 43, 55]

In [6]:
def trade_within_hours(row):
    exchange = row["exchange"]
    
    if exchange in exchange_info.keys():
        trade_start = datetime.strptime(row["start_time"], "%H:%M").time()
        trade_end = datetime.strptime(row["end_time"], "%H:%M").time()
        
        open = datetime.strptime(exchange_info[exchange]["start"], "%H:%M").time()
        close = datetime.strptime(exchange_info[exchange]["end"], "%H:%M").time()
        
        if trade_start > open and trade_end < close:
            return True
        else: 
            return False
    else:
        print(f"exchange '{exchange}' not in dict")

In [7]:
example_df["within_hours"] = example_df.apply(trade_within_hours, axis=1)

exchange 'BZ' not in dict


In [8]:
example_df

,exchange,start_time,end_time,price,avg_price,within_hours
0,LN,09:34,09:39,452,45,True
1,LN,08:59,21:59,145,2,False
2,US,10:44,10:48,34,45,False
3,LN,12:44,12:34,12,12,True
4,US,9:34,11:34,4,5,False
5,BZ,05:44,09:55,43,55,None


---

In [9]:
exchange_map = {}

for key, value in exchange_info.items():
    exchange_map[key] = value["region"]
    
exchange_map

{'LN': 'WE', 'US': 'NA'}

In [10]:
example_df["region"] = example_df["exchange"].map(exchange_map)

example_df

,exchange,start_time,end_time,price,avg_price,within_hours,region
0,LN,09:34,09:39,452,45,True,WE
1,LN,08:59,21:59,145,2,False,WE
2,US,10:44,10:48,34,45,False,NA
3,LN,12:44,12:34,12,12,True,WE
4,US,9:34,11:34,4,5,False,NA
5,BZ,05:44,09:55,43,55,None,NaN


---

In [11]:
exchange_rate = {}

for key, value in exchange_info.items():
    exchange_rate[key] = value["exchange_rate"]
    
exchange_rate

{'LN': 1.2853, 'US': 1}

In [12]:
pre_multi = example_df.copy()

In [13]:
price_columns = ["price", "avg_price"]

for price_column in price_columns:
    example_df[price_column] = example_df["exchange"].map(exchange_rate) * example_df[price_column]

In [14]:
exchange_rate

{'LN': 1.2853, 'US': 1}

In [15]:
pre_multi

,exchange,start_time,end_time,price,avg_price,within_hours,region
0,LN,09:34,09:39,452,45,True,WE
1,LN,08:59,21:59,145,2,False,WE
2,US,10:44,10:48,34,45,False,NA
3,LN,12:44,12:34,12,12,True,WE
4,US,9:34,11:34,4,5,False,NA
5,BZ,05:44,09:55,43,55,None,NaN


In [16]:
example_df

,exchange,start_time,end_time,price,avg_price,within_hours,region
0,LN,09:34,09:39,580.9556,57.8385,True,WE
1,LN,08:59,21:59,186.3685,2.5706,False,WE
2,US,10:44,10:48,34.0000,45.0000,False,NA
3,LN,12:44,12:34,15.4236,15.4236,True,WE
4,US,9:34,11:34,4.0000,5.0000,False,NA
5,BZ,05:44,09:55,NaN,NaN,None,NaN


## Action Plan for 11/10/2023
- Create .py file containing a dictionary which holds exchange information
- Use the .apply() function to map all of the regions, exchange rates, and whether the trade was within the opening/close hours
- Investigate the use of trade_value and qty_executed as the weights for analysis